In [10]:
# Install necessary libraries
!pip install google-generativeai PyPDF2 python-docx docx2txt langchain langchain-google-genai

INFO: pip is looking at multiple versions of langchain-google-genai to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.4 MB/s eta 0:00:00


In [11]:
import google.generativeai as genai
import PyPDF2
import docx2txt
from docx import Document
import io
import os
from typing import List, Dict
import json
import re
from google.colab import files

In [12]:
# Configure Gemini API using Colab Secrets
from google.colab import userdata

try:
    # Get API key from Colab secrets
    API_KEY = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=API_KEY)

    # Initialize the model with Gemini 1.5 Flash (free tier)
    model = genai.GenerativeModel('gemini-1.5-flash')

    print("✅ Gemini API configured successfully with gemini-1.5-flash!")
    print("✅ API key loaded from Colab secrets")

except Exception as e:
    print(f"❌ Error: {e}")
    print("\n🔧 Setup Instructions:")
    print("1. Go to the left sidebar in Colab")
    print("2. Click on the 🔑 key icon (Secrets)")
    print("3. Click 'Add new secret'")
    print("4. Name: GOOGLE_API_KEY")
    print("5. Value: Your Gemini API key from https://aistudio.google.com/app/apikey")
    print("6. Toggle 'Notebook access' ON")
    print("7. Re-run this cell")

✅ Gemini API configured successfully with gemini-1.5-flash!
✅ API key loaded from Colab secrets


In [14]:
class StudyAssistant:
    def __init__(self, model):
        self.model = model

    def extract_text_from_pdf(self, pdf_file):
        """Extract text from uploaded PDF file"""
        try:
            reader = PyPDF2.PdfReader(pdf_file)
            text = ""
            for page in reader.pages:
                text += page.extract_text() + "\n"
            return text
        except Exception as e:
            print(f"Error reading PDF: {e}")
            return None

    def extract_text_from_docx(self, docx_file):
        """Extract text from uploaded Word document"""
        try:
            # Method 1: Using docx2txt (simpler and more reliable)
            text = docx2txt.process(docx_file)
            if text:
                return text

            # Method 2: Using python-docx as fallback
            docx_file.seek(0)  # Reset file pointer
            doc = Document(docx_file)
            text = ""
            for paragraph in doc.paragraphs:
                text += paragraph.text + "\n"

            # Also extract text from tables
            for table in doc.tables:
                for row in table.rows:
                    for cell in row.cells:
                        text += cell.text + " "
                    text += "\n"

            return text
        except Exception as e:
            print(f"Error reading Word document: {e}")
            return None

    def detect_file_type(self, filename):
        """Detect file type based on extension"""
        filename = filename.lower()
        if filename.endswith('.pdf'):
            return 'pdf'
        elif filename.endswith(('.docx', '.doc')):
            return 'docx'
        else:
            return 'unknown'

    def extract_text_from_file(self, file_content, filename):
        """Extract text from any supported file format"""
        file_type = self.detect_file_type(filename)
        file_obj = io.BytesIO(file_content)

        if file_type == 'pdf':
            return self.extract_text_from_pdf(file_obj)
        elif file_type == 'docx':
            return self.extract_text_from_docx(file_obj)
        else:
            print(f"❌ Unsupported file type: {filename}")
            print("📋 Supported formats: PDF (.pdf), Word (.docx, .doc)")
            return None

    def summarize_content(self, content: str) -> str:
        """Summarize the study material into key points"""
        prompt = f"""
Please summarize the following study material into concise bullet points that capture the key concepts and important information:

Study Material:
{content}

Instructions:
- Create 5-8 clear, concise bullet points
- Focus on the most important concepts
- Each point should be 1-2 sentences maximum
- Use simple, clear language

Summary:
"""

        try:
            response = self.model.generate_content(prompt)
            return response.text
        except Exception as e:
            print(f"Error generating summary: {e}")
            return "Unable to generate summary"

    def generate_quiz_questions(self, content: str, num_questions: int = 5) -> List[Dict]:
        """Generate multiple-choice quiz questions based on the content"""
        prompt = f"""
Based on the following study material, create {num_questions} multiple-choice quiz questions.

Study Material:
{content}

Instructions:
- Create {num_questions} multiple-choice questions
- Each question should have 4 options (a, b, c, d)
- Questions should test understanding of key concepts
- Include the correct answer for each question
- Vary the difficulty from basic recall to application
- Format each question clearly

Please format your response as follows for each question:
Question X: [Question text]
a) [Option A]
b) [Option B]
c) [Option C]
d) [Option D]
Correct Answer: [Letter]
Explanation: [Brief explanation]

Quiz Questions:
"""

        try:
            response = self.model.generate_content(prompt)
            return self._parse_quiz_response(response.text)
        except Exception as e:
            print(f"Error generating quiz: {e}")
            return []

    def _parse_quiz_response(self, response_text: str) -> List[Dict]:
        """Parse the quiz response into structured format"""
        questions = []
        question_blocks = re.split(r'Question \d+:', response_text)[1:]  # Skip first empty element

        for i, block in enumerate(question_blocks, 1):
            try:
                lines = block.strip().split('\n')
                question_text = lines[0].strip()

                options = {}
                correct_answer = ""
                explanation = ""

                for line in lines[1:]:
                    line = line.strip()
                    if line.startswith(('a)', 'b)', 'c)', 'd)')):
                        key = line[0]
                        value = line[3:].strip()
                        options[key] = value
                    elif line.startswith('Correct Answer:'):
                        correct_answer = line.split(':')[1].strip().lower()
                    elif line.startswith('Explanation:'):
                        explanation = line.split(':', 1)[1].strip()

                if question_text and options and correct_answer:
                    questions.append({
                        'question': question_text,
                        'options': options,
                        'correct_answer': correct_answer,
                        'explanation': explanation
                    })
            except Exception as e:
                print(f"Error parsing question {i}: {e}")
                continue

        return questions

    def display_quiz(self, questions: List[Dict]):
        """Display quiz questions in a user-friendly format"""
        print("=" * 60)
        print("📚 GENERATED QUIZ QUESTIONS")
        print("=" * 60)

        for i, q in enumerate(questions, 1):
            print(f"\n🔹 Question {i}: {q['question']}")
            print()
            for key, value in q['options'].items():
                print(f"   {key}) {value}")
            print(f"\n✅ Correct Answer: {q['correct_answer'].upper()}")
            if q['explanation']:
                print(f"💡 Explanation: {q['explanation']}")
            print("-" * 50)

# Initialize the Study Assistant
study_assistant = StudyAssistant(model)
print("✅ Study Assistant initialized successfully!")
print("📋 Supported file formats: PDF (.pdf), Word (.docx, .doc)")

✅ Study Assistant initialized successfully!
📋 Supported file formats: PDF (.pdf), Word (.docx, .doc)


In [15]:
def upload_and_process_file():
    """Upload and process PDF or Word document"""
    print("📁 Please upload your study material:")
    print("📋 Supported formats: PDF (.pdf), Word (.docx, .doc)")

    uploaded = files.upload()

    if not uploaded:
        print("❌ No file uploaded")
        return None, None

    filename = list(uploaded.keys())[0]
    file_content = uploaded[filename]

    print(f"✅ File uploaded: {filename}")
    print(f"📊 File size: {len(file_content)} bytes")

    # Detect and process the file
    text = study_assistant.extract_text_from_file(file_content, filename)

    if text:
        text = text.strip()
        if len(text) < 50:
            print("⚠️  Warning: Extracted text is very short. Please check if the file contains readable text.")

        word_count = len(text.split())
        print(f"✅ Successfully extracted {len(text)} characters ({word_count} words)")

        # Show preview of extracted text
        preview = text[:200] + "..." if len(text) > 200 else text
        print(f"\n📖 Text preview:\n{preview}")

        return text, filename
    else:
        print("❌ Failed to extract text from file")
        print("🔧 Troubleshooting tips:")
        print("   - Ensure the file is not password protected")
        print("   - Check if the file contains readable text (not just images)")
        print("   - Try a different file format")
        return None, None

def process_text_input():
    """Process text input directly"""
    print("📝 Enter your study material (paste text below):")
    print("💡 Tip: You can paste content from any source")
    print("Type 'END' on a new line when finished:")
    print("-" * 40)

    lines = []
    while True:
        try:
            line = input()
            if line.strip().upper() == 'END':
                break
            lines.append(line)
        except KeyboardInterrupt:
            print("\n❌ Input cancelled")
            return None

    text = '\n'.join(lines).strip()
    if text:
        word_count = len(text.split())
        print(f"✅ Successfully captured {len(text)} characters ({word_count} words)")
        return text
    else:
        print("❌ No text provided")
        return None

def validate_content(text):
    """Validate if the content is suitable for quiz generation"""
    if not text or len(text.strip()) < 50:
        print("⚠️  Content is too short for effective quiz generation.")
        print("💡 Please provide at least a few sentences of study material.")
        return False

    word_count = len(text.split())
    if word_count < 20:
        print("⚠️  Content has very few words. Quiz quality may be limited.")
        return False

    return True

print("✅ Enhanced upload and processing functions ready!")
print("📋 Supported formats: PDF, Word documents, and direct text input")

✅ Enhanced upload and processing functions ready!
📋 Supported formats: PDF, Word documents, and direct text input


In [16]:
def run_study_assistant():
    """Main function to run the study assistant"""
    print("🎓 Welcome to the Study Assistant Quiz Generator!")
    print("=" * 60)

    # Choose input method
    print("\nChoose your input method:")
    print("1. Upload PDF file")
    print("2. Enter text directly")

    choice = input("\nEnter your choice (1 or 2): ").strip()

    study_material = None

    if choice == '1':
        study_material, filename = upload_and_process_file()
    elif choice == '2':
        study_material = process_text_input()
    else:
        print("❌ Invalid choice. Please run again.")
        return

    if not study_material:
        print("❌ No study material to process. Please try again.")
        return

    print("\n" + "=" * 60)
    print("📋 GENERATING SUMMARY...")
    print("=" * 60)

    # Generate summary
    summary = study_assistant.summarize_content(study_material)
    print(summary)

    # Ask for number of questions
    while True:
        try:
            num_questions = int(input(f"\n🤔 How many quiz questions would you like? (1-10): "))
            if 1 <= num_questions <= 10:
                break
            else:
                print("Please enter a number between 1 and 10.")
        except ValueError:
            print("Please enter a valid number.")

    print(f"\n⏳ Generating {num_questions} quiz questions...")

    # Generate quiz questions
    questions = study_assistant.generate_quiz_questions(study_material, num_questions)

    if questions:
        study_assistant.display_quiz(questions)

        # Option to save results
        save_choice = input(f"\n💾 Would you like to save the quiz to a file? (y/n): ").strip().lower()
        if save_choice == 'y':
            save_quiz_to_file(summary, questions)
    else:
        print("❌ Failed to generate quiz questions. Please try again.")

def save_quiz_to_file(summary, questions):
    """Save quiz results to a text file"""
    filename = f"quiz_results.txt"

    with open(filename, 'w', encoding='utf-8') as f:
        f.write("STUDY MATERIAL SUMMARY\n")
        f.write("=" * 50 + "\n")
        f.write(summary + "\n\n")

        f.write("QUIZ QUESTIONS\n")
        f.write("=" * 50 + "\n")

        for i, q in enumerate(questions, 1):
            f.write(f"Question {i}: {q['question']}\n")
            for key, value in q['options'].items():
                f.write(f"   {key}) {value}\n")
            f.write(f"Correct Answer: {q['correct_answer'].upper()}\n")
            if q['explanation']:
                f.write(f"Explanation: {q['explanation']}\n")
            f.write("-" * 30 + "\n")

    print(f"✅ Quiz saved to {filename}")
    files.download(filename)

print("✅ Main application interface ready!")


✅ Main application interface ready!


In [17]:
# Run the Study Assistant
run_study_assistant()

🎓 Welcome to the Study Assistant Quiz Generator!

Choose your input method:
1. Upload PDF file
2. Enter text directly

Enter your choice (1 or 2): 1
📁 Please upload your study material:
📋 Supported formats: PDF (.pdf), Word (.docx, .doc)


Saving Day-2-business-report.pdf to Day-2-business-report.pdf
✅ File uploaded: Day-2-business-report.pdf
📊 File size: 419957 bytes
✅ Successfully extracted 38229 characters (4005 words)

📖 Text preview:
Assignment
 
of
 
Prompt
 
Engineering
 
 
Topics
 
:
 
“
Create
 
and
 
Validate
 
Your
 
Business
 
Plan
 
with
 
Gener ative
 
AI
”
 
 
Date:
 
10-06-2025
 
 
AI
 
Agentic
 
:
 
Perplexity
 
 
Prom...

📋 GENERATING SUMMARY...
* The study focuses on creating and validating low-budget AI healthcare startup ideas for Tier-2 Indian cities.  These leverage generative AI and require only basic tech skills.

* Three AI healthcare business ideas were validated: a multilingual mental health chatbot, an AI-powered personalized meal planner for chronic patients, and an AI medical transcription service for clinics.

* Validation considered problem-solution fit, market demand (using data from NASSCOM, FICCI, etc.), existing competition, differentiation strategies, and low-budget feasibility.

* A

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Test with sample content from the course document
sample_content = """
Prompt engineering involves designing and refining inputs to language models to achieve desired outputs. In the context of agents, prompt engineering allows for better control over how an agent interacts with the environment and solves specific tasks. This is particularly useful in domains like robotics and conversational AI. By adjusting the structure and content of the prompts, users can enhance an agent's performance on specific tasks.

Key aspects of prompt engineering include:
- Understanding the model's capabilities and limitations
- Crafting clear and specific instructions
- Using examples to guide model behavior
- Iterative refinement based on outputs
- Context management for better results

Effective prompt engineering can significantly improve the quality and relevance of AI-generated responses, making it an essential skill for working with language models in various applications.
"""

print("🧪 Testing with sample content...")
print("\n" + "=" * 60)
print("📋 SAMPLE SUMMARY:")
print("=" * 60)

summary = study_assistant.summarize_content(sample_content)
print(summary)

print("\n⏳ Generating sample quiz questions...")
questions = study_assistant.generate_quiz_questions(sample_content, 3)

if questions:
    study_assistant.display_quiz(questions)
else:
   print("❌ Failed to generate sample questions")

🧪 Testing with sample content...

📋 SAMPLE SUMMARY:
* Prompt engineering designs input instructions for language models to get desired outputs.
* It's crucial for controlling AI agents in tasks, especially in robotics and conversational AI.
* Effective prompts require understanding the model's abilities and weaknesses.
* Clear, specific instructions and illustrative examples guide the model's actions.
* Iterative refinement based on the model's responses is essential for improvement.
* Context is important for better, more relevant results.
* Well-crafted prompts significantly enhance AI response quality and relevance.
* Prompt engineering is a vital skill for working with language models across applications.


⏳ Generating sample quiz questions...
📚 GENERATED QUIZ QUESTIONS

🔹 Question 1: What is the primary goal of prompt engineering in the context of AI agents?

   a) To increase the speed of the AI agent's processing.
   b) To improve the AI agent's interaction with the environment